In [2]:
from ultralytics import YOLO
import os
import yaml
from IPython.display import Image
import json
import cv2
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import torch
import pandas as pd
import random
import easyocr
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from tqdm import tqdm

In [3]:
project_dir = os.path.dirname(os.getcwd())
dataset_location = os.path.join(project_dir, 'data', 'ocr_evaluation')
training_name = 'experiment3'
log_dir = os.path.join(project_dir, 'data', 'roboflow_dataset_yolo_v11', 'logs')
model_weights_location = f'{log_dir}/{training_name}/weights/best.pt'

output_location = os.path.join(project_dir, 'data', 'yolo_v11')


In [4]:
model = YOLO(model_weights_location)

# Specify the directory containing images to detect objects on
images_dir = dataset_location  # replace with the path to your dataset
output_log_dir = f'{output_location}/detection_results/labels'  # directory to save detection info
output_img_dir = f'{output_location}/detection_results'
cropped_plates_path = f'{output_location}/cropped_plates'

Path(output_log_dir).mkdir(parents=True, exist_ok=True)
Path(output_img_dir).mkdir(parents=True, exist_ok=True)
Path(cropped_plates_path).mkdir(parents=True, exist_ok=True)

In [5]:
# Run inference and save details
for image_path in Path(images_dir).glob("*.jpg"):  # replace with the appropriate extension if different
    # Perform detection
    results = model(image_path)
    
    # Prepare to save results
    detections = []
    # Process results list
    # for result in results:
    #     boxes = result.boxes  # Boxes object for bounding box outputs
    #     masks = result.masks  # Masks object for segmentation masks outputs
    #     keypoints = result.keypoints  # Keypoints object for pose outputs
    #     probs = result.probs  # Probs object for classification outputs
    #     obb = result.obb  # Oriented boxes object for OBB outputs
    #     result.show()  # display to screen
    #     result.save(filename="result.jpg")  # save to disk
    max_conf = -1.0
    for det in results[0].boxes.data:  # detections for the image
        conf = float(det[4])
        if conf < max_conf:
            continue
        else:
            max_conf = conf
        x1, y1, x2, y2, conf, cls = map(float, det[:6])
        # class_name = model.names[int(cls)]
        detections.append({
            "x1": x1,
            "y1": y1,
            "x2": x2,
            "y2": y2,
            "confidence": conf,
            "class": int(cls)
        })
    
    if max_conf == -1:
        continue

    img = cv2.imread(image_path)
    cropped_plate = img[int(y1):int(y2), int(x1):int(x2)]
    plate_save_path = os.path.join(cropped_plates_path, f'plate_{image_path.stem}.jpg')
    cv2.imwrite(plate_save_path, cropped_plate)

    results[0].save(filename=f'{output_img_dir}/{image_path.stem}.jpg')
    
    # Save detections to a JSON file
    output_file = Path(output_log_dir) / f"{image_path.stem}_detections.json"
    with open(output_file, 'w') as f:
        json.dump(detections, f, indent=4)



image 1/1 /home/roma/5sem/Real-time_Number_Plate_Recognition/data/ocr_evaluation/00298.jpg: 480x640 1 License_Plate, 48.2ms
Speed: 3.2ms preprocess, 48.2ms inference, 92.4ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/roma/5sem/Real-time_Number_Plate_Recognition/data/ocr_evaluation/00765.jpg: 480x640 1 License_Plate, 7.0ms
Speed: 2.1ms preprocess, 7.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/roma/5sem/Real-time_Number_Plate_Recognition/data/ocr_evaluation/01508.jpg: 448x640 1 License_Plate, 48.6ms
Speed: 2.0ms preprocess, 48.6ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/roma/5sem/Real-time_Number_Plate_Recognition/data/ocr_evaluation/01348.jpg: 448x640 1 License_Plate, 6.5ms
Speed: 2.8ms preprocess, 6.5ms inference, 3.0ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/roma/5sem/Real-time_Number_Plate_Recognition/data/ocr_evaluation/00844.jpg: 480x640 1 License_Plat